# 部分固有値と付随する固有ベクトル（部分固有対）の精度保証付き数値計算

MATLABの区間演算パッケージである[INTLAB](https://www.tuhh.de/ti3/rump/intlab/)には一般化固有値問題の固有値・固有ベクトルを精度保証付きで計算する`verifyeig`という関数が実装されている。本稿では、文献1および2を参考に`verifyeig`関数のアルゴリズムを説明し、一般化固有値問題 

$$
Ax=\lambda Bx
$$

を満たす固有値 $\lambda\in\mathbb{C}$ と固有ベクトル $x\in\mathbb{C}^n$ ($n$ は行列の次元) を求める実装を紹介する。

行列 $A,B\in \mathbb{C}^{n\times n}$ を与え、$B$ は正則、$B^{-1}A$ が対角化可能、かつ全ての固有値 $\lambda$ が重複固有値でないと仮定する。写像 $F:\mathbb{C}^{n+1}\to\mathbb{C}^{n+1}$ を

$$
    F(\lambda, x) := \begin{bmatrix}x^Hx -1\\ Ax-\lambda Bx\end{bmatrix}
$$

と定義すると $F(\lambda, x)=0$ をみたす $(\lambda, x)$ は一般化固有値問題の固有対となる（固有ベクトルは $\|x\|_2=1$ と正規化されている）。いま $\bar x\in \mathbb{C}^n$, $\bar \lambda \in \mathbb{C}$ を $F(\bar\lambda,\bar x)\approx 0$ とする近似解とし、
$R\in \mathbb{C}^{n+1\times n+1}$ を写像 $F$ の近似解におけるヤコビ行列の近似逆行列（$R\approx DF(\bar\lambda,\bar x)^{-1}$）とする。部分固有値・固有ベクトルの精度保証付き数値計算には次の定理を利用する。

**定理** $\boldsymbol{y}\in \mathbb{IC}^n, \boldsymbol{d} \in \mathbb{IC}$ をある区間ベクトルと区間として、$\boldsymbol{w}:=(\boldsymbol{d},\boldsymbol{y}^T)^T\in \mathbb{IC}^{n+1}$ とする（${}^T$はベクトルの単なる転置を意味する）。 さらに

$$
    g(\boldsymbol{w}):=z+\left(I-R\cdot DF\left(\bar\lambda + \boldsymbol{d}, \bar x + \boldsymbol{y}\right)\right)\boldsymbol{w}
$$

と定義する。上の式のヤコビ行列 $DF(\lambda,x)$ と $z$ は

$$
\begin{align*}
DF(\lambda,x):=
\begin{bmatrix}
0 & 2x^T\\
-B\boldsymbol{x} & A-\boldsymbol{\lambda}B \\
\end{bmatrix},\quad
z:=-R
\left[\begin{array}
a\bar x_1^2+\bar x_2^2+\dots +\bar x_n^2-1 \\
A\bar x-\bar\lambda B\bar x \\
\end{array}\right]
\end{align*}
$$

で与えられる。
このとき $\mathrm{int}(\boldsymbol{w})$ を区間ベクトル $\boldsymbol{w}$ の（要素ごとの）内部とすると、$g(\boldsymbol{w})\subset \mathrm{int}(\boldsymbol{w})$ ならば、一般化固有値問題 $Ax=\lambda Bx$ の真の固有値 $\lambda$ が
$\bar \lambda +\boldsymbol{d}$ 内に唯一存在し、対応する固有ベクトルが $\bar x + \boldsymbol{y}$ に包含される。

この定理をもとに精度保証付き数値計算を次のように実装する。

1. 近似固有値 $\bar{\lambda}$、近似固有ベクトル $\bar{x}$ を入力し、これらが実数であるときは実数の区間・区間ベクトルで、いずれかが複素数であるときは複素数の区間・区間ベクトルで $\boldsymbol{y}$, $\boldsymbol{d}$ を定義し、候補ベクトル $\boldsymbol{w}=(\boldsymbol{d},\boldsymbol{y}^T)^T$ を作成する。
1. ヤコビ行列$DF$ と $z$、$g(\boldsymbol{w}):=z+\left(I-R\cdot DF\left(\bar\lambda + \boldsymbol{d}, \bar x + \boldsymbol{y}\right)\right)\boldsymbol{w}$ を区間演算により計算する。
1. $g(\boldsymbol{w})\subsetneq \boldsymbol{w}$を満たすとき、$(\bar\lambda,\bar x)^T+g(\boldsymbol{w})$ の値を返す。

上記の手順に従って次元 $n=30$ のある行列 $A,B\in \mathbb{C}^{n\times n}$ に対し、$Ax=\lambda Bx$ を考える。
まず近似固有値$\bar{\lambda}$、近似固有ベクトル$\bar{x}$を計算する。

In [1]:
versioninfo()

Julia Version 1.10.0
Commit 3120989f39b (2023-12-25 18:01 UTC)
Build Info:
  Official https://julialang.org/ release
Platform Info:
  OS: macOS (arm64-apple-darwin22.4.0)
  CPU: 14 × Apple M3 Max
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-15.0.7 (ORCJIT, apple-m1)
  Threads: 2 on 10 virtual cores


In [2]:
using Pkg
Pkg.status("IntervalArithmetic")

Status `~/.julia/environments/v1.10/Project.toml`
  [d1acc4aa] IntervalArithmetic v0.22.6


In [3]:
using LinearAlgebra
n = 30
A, B = randn(n, n), randn(n, n)
λ, x = eigen(A, B)

GeneralizedEigen{ComplexF64, ComplexF64, Matrix{ComplexF64}, Vector{ComplexF64}}
values:
30-element Vector{ComplexF64}:
   -8.807235191171742 + 0.0im
  -1.8720739353881435 - 0.4211957322307501im
  -1.8720739353881435 + 0.42119573223075013im
  -1.2450164394864036 - 2.338661769764113im
  -1.2450164394864036 + 2.338661769764113im
  -1.1558402670064905 + 0.0im
   -0.789551856115187 - 1.002224359974505im
   -0.789551856115187 + 1.002224359974505im
  -0.5393502183046783 - 0.7116749651587355im
  -0.5393502183046783 + 0.7116749651587354im
  -0.3675440335623449 + 0.0im
 -0.22852573744671337 - 0.1592805432423365im
 -0.22852573744671337 + 0.15928054324233654im
                      ⋮
    0.281692718115899 - 0.545904155982315im
    0.281692718115899 + 0.5459041559823149im
   0.2965924537980811 - 0.8603906140438535im
   0.2965924537980811 + 0.8603906140438535im
   0.5212264660922051 + 0.0im
   0.8420124984325313 - 0.4056142428796466im
   0.8420124984325315 + 0.4056142428796465im
   0.86681469978153

1. 近似固有値 $\bar{\lambda}$、近似固有ベクトル $\bar{x}$ を入力し、これらが実数であるときは実数の区間・区間ベクトルで、いずれかが複素数であるときは複素数の区間・区間ベクトルで $\boldsymbol{y}$, $\boldsymbol{d}$ を定義し、候補ベクトル $\boldsymbol{w}=(\boldsymbol{d},\boldsymbol{y}^T)^T$ を作成する。

候補ベクトルの区間幅は大きすぎたり、小さすぎたりすると定理の検証が失敗する。そのため**ある程度**小さい大きさを選ぶ必要がある。今回の実装では $\epsilon=10^{-9}$ とした。

In [4]:
using IntervalArithmetic
lam = λ[1]
x1 = x[:, 1] # 対応する固有ベクトル
x = x1 ./ sqrt(x1' * x1)
ysize = length(x)

ϵ = 1e-9 # size of candidate vector

if isreal(lam) && isreal(x)
    lam = real(lam)
    x = real(x)
    id = interval(0, ϵ; format=:midpoint)
    iy = interval.(zeros(ysize), ϵ; format=:midpoint)
    iI = interval(Matrix{Float64}(I, n + 1, n + 1))
    izero = interval(0)
else
    id = Complex(interval.(0, ϵ; format=:midpoint), interval.(0, ϵ; format=:midpoint))
    iy = Complex.(interval.(zeros(ysize), ϵ; format=:midpoint), interval.(zeros(ysize), ϵ; format=:midpoint))
    iI = interval(Matrix{Complex{Float64}}(I, n + 1, n + 1))
    izero = interval(0 + 0im)
end
iw = [id; iy]

31-element Vector{Interval{Float64}}:
 [-1.00001e-09, 1.00001e-09]_com
 [-1.00001e-09, 1.00001e-09]_com
 [-1.00001e-09, 1.00001e-09]_com
 [-1.00001e-09, 1.00001e-09]_com
 [-1.00001e-09, 1.00001e-09]_com
 [-1.00001e-09, 1.00001e-09]_com
 [-1.00001e-09, 1.00001e-09]_com
 [-1.00001e-09, 1.00001e-09]_com
 [-1.00001e-09, 1.00001e-09]_com
 [-1.00001e-09, 1.00001e-09]_com
 [-1.00001e-09, 1.00001e-09]_com
 [-1.00001e-09, 1.00001e-09]_com
 [-1.00001e-09, 1.00001e-09]_com
   ⋮
 [-1.00001e-09, 1.00001e-09]_com
 [-1.00001e-09, 1.00001e-09]_com
 [-1.00001e-09, 1.00001e-09]_com
 [-1.00001e-09, 1.00001e-09]_com
 [-1.00001e-09, 1.00001e-09]_com
 [-1.00001e-09, 1.00001e-09]_com
 [-1.00001e-09, 1.00001e-09]_com
 [-1.00001e-09, 1.00001e-09]_com
 [-1.00001e-09, 1.00001e-09]_com
 [-1.00001e-09, 1.00001e-09]_com
 [-1.00001e-09, 1.00001e-09]_com
 [-1.00001e-09, 1.00001e-09]_com

2. ヤコビ行列$DF$ と $z$、$g(\boldsymbol{w}):=z+\left(I-R\cdot DF\left(\bar\lambda + \boldsymbol{d}, \bar x + \boldsymbol{y}\right)\right)\boldsymbol{w}$ を区間演算により計算する。

In [5]:
ix = interval(x);
ilam = interval(lam);
iA = interval(A);
iB = interval(B);
iDF = [izero transpose(interval(2) * (ix + iy)); -iB*(ix+iy) iA-(ilam.+id)*iB]
R = inv(mid.(iDF));
iR = interval(R);
z = -iR * [dot(ix, ix) - interval(1); iA * ix - ilam * iB * ix]
gw = z + (iI - iR * iDF) * iw

31-element Vector{Interval{Float64}}:
 [-1.53421e-12, 1.60364e-12]_com
 [-6.28847e-15, 5.87119e-15]_com
 [-6.2841e-15, 6.24573e-15]_com
 [-1.33737e-14, 1.35622e-14]_com
 [-5.34187e-15, 5.95928e-15]_com
 [-1.65682e-14, 1.55644e-14]_com
 [-4.44868e-15, 4.00426e-15]_com
 [-2.1826e-14, 2.10103e-14]_com
 [-5.66461e-15, 6.17318e-15]_com
 [-9.08141e-15, 8.89336e-15]_com
 [-1.68061e-14, 1.59398e-14]_com
 [-3.91177e-15, 3.74838e-15]_com
 [-3.26079e-15, 3.35143e-15]_com
   ⋮
 [-3.99718e-15, 4.32441e-15]_com
 [-9.42529e-15, 9.6057e-15]_com
 [-1.38155e-14, 1.34268e-14]_com
 [-1.58001e-14, 1.49484e-14]_com
 [-3.53401e-15, 3.64503e-15]_com
 [-1.14521e-14, 1.23865e-14]_com
 [-2.50687e-14, 2.35591e-14]_com
 [-1.37185e-14, 1.32005e-14]_com
 [-2.83772e-14, 2.69667e-14]_com
 [-2.06638e-14, 2.20965e-14]_com
 [-1.31275e-14, 1.3718e-14]_com
 [-1.19503e-14, 1.15808e-14]_com

3. $g(\boldsymbol{w})\subsetneq \boldsymbol{w}$を満たすとき、$(\bar\lambda,\bar x)^T+g(\boldsymbol{w})$ の値を返す。

In [6]:
all(issubset_interval.(gw, iw))

true

In [7]:
[ilam; ix] + gw

31-element Vector{Interval{Float64}}:
 [-8.80724, -8.80723]_com
 [0.121617, 0.121618]_com
 [-0.130802, -0.130801]_com
 [-0.601561, -0.60156]_com
 [-0.162248, -0.162246]_com
 [-0.0338032, -0.0338031]_com
 [0.15269, 0.152691]_com
 [-0.100636, -0.100635]_com
 [-0.193449, -0.193448]_com
 [-0.209609, -0.209608]_com
 [-0.112811, -0.11281]_com
 [-0.0437788, -0.0437787]_com
 [-0.264069, -0.264068]_com
   ⋮
 [-0.00728467, -0.00728466]_com
 [-0.0881746, -0.0881745]_com
 [-0.187405, -0.187404]_com
 [0.00250188, 0.00250189]_com
 [0.13498, 0.134981]_com
 [-0.124575, -0.124574]_com
 [0.186956, 0.186958]_com
 [-0.178369, -0.178368]_com
 [-0.325139, -0.325138]_com
 [-0.205561, -0.205559]_com
 [0.146769, 0.14677]_com
 [-0.0235999, -0.0235998]_com

この第一成分は対象の固有値の厳密な包含である。

In [8]:
ilam + gw[1]

[-8.80724, -8.80723]_com

この手順を`verifyeig関数`として定義する。

In [9]:
using IntervalArithmetic
function verifyeig(A, lam, x, B=B = Matrix(I,size(A)))
    x = x ./ sqrt(x' * x)
    ysize = length(x)

    ϵ = 1e-9 # size of candidate vector

    if isreal(lam) && isreal(x)
        lam = real(lam)
        x = real(x)
        id = interval(0, ϵ; format=:midpoint)
        iy = interval.(zeros(ysize), ϵ; format=:midpoint)
        iI = interval(Matrix{Float64}(I, ysize + 1, ysize + 1))
        izero = interval(0)
    else
        id = Complex(interval.(0, ϵ; format=:midpoint), interval.(0, ϵ; format=:midpoint))
        iy = Complex.(interval.(zeros(ysize), ϵ; format=:midpoint), interval.(zeros(ysize), ϵ; format=:midpoint))
        iI = interval(Matrix{Complex{Float64}}(I, ysize + 1, ysize + 1))
        izero = interval(0 + 0im)
    end
    iw = [id; iy]

    # DF(w) = [0 transpose(2*(x+w[2:end])) ; -B*(x+w[2:end]) A-(lam+w[1]).*B]
    ix = interval(x)
    ilam = interval(lam)
    iA = interval(A)
    iB = interval(B)

    iDF(w) = [izero transpose(interval(2) * (ix + w[2:end])); -iB*(ix+w[2:end]) iA-(ilam.+w[1])*iB]
    R = inv(mid.(iDF(zeros(ysize + 1))))
    iR = interval(R)
    z = -iR * [dot(ix, ix) - interval(1); iA * ix - ilam * iB * ix]
    g(w) = z + (iI - iR * iDF(w)) * w
    gw = g(iw)
    if all(issubset_interval.(gw, iw))
        while maximum(radius, gw) / norm([lam; x], 1) > 5e-13
            iw = gw
            gw = g(iw)
        end
        return ilam + gw[1]
    else
        return NaN
    end
end


verifyeig (generic function with 2 methods)

部分固有対の精度保証方法であるので、全固有値を精度保証するには向かない。以下のように全固有値を精度保証するには $O(n^4)$ のオーダーが必要になる。全固有値を精度保証するには、例えば、[標準固有値問題の精度保証付き数値解法](https://www.risk.tsukuba.ac.jp/~takitoshi/tutorial/verifyalleig.html)のような方法が有効である。

In [10]:
using LinearAlgebra
n = 30
A, B = randn(n, n), randn(n, n)
λ, x = eigen(A, B)

GeneralizedEigen{ComplexF64, ComplexF64, Matrix{ComplexF64}, Vector{ComplexF64}}
values:
30-element Vector{ComplexF64}:
   -3.927248809102562 + 0.0im
   -2.342175403774986 + 0.0im
  -1.8204436211401438 + 0.0im
  -0.9825359650919164 + 0.9062641005675697im
  -0.9825359650919163 - 0.9062641005675698im
  -0.9369434243815118 + 0.0im
  -0.7678548617673059 + 0.0im
  -0.6336042505433659 - 4.4581246542775945im
  -0.6336042505433657 + 4.4581246542775945im
 -0.41284829356200076 + 0.0im
  -0.3207959087791363 + 0.0im
 -0.26666982104990433 - 0.36793157410745986im
 -0.26666982104990433 + 0.36793157410745986im
                      ⋮
 -0.04156652673607428 + 0.8154422472066382im
   0.2577455745317516 - 0.18455474115985465im
   0.2577455745317516 + 0.18455474115985465im
   0.6689537858084744 - 0.24860338555344522im
   0.6689537858084744 + 0.24860338555344522im
   0.7723122085046161 + 0.6492318019823247im
   0.7723122085046162 - 0.6492318019823247im
   1.1473309779693959 - 0.10819027220057749im
   1.1473

In [11]:
# ilam = zeros(Interval,length(λ))
for i = 1:length(λ)
    @show ilam = verifyeig(A, λ[i], x[:, i], B)
end

ilam = verifyeig(A, λ[i], x[:, i], B) = Interval{Float64}(-3.9272488091026454, -3.9272488091024935, com)
ilam = verifyeig(A, λ[i], x[:, i], B) = Interval{Float64}(-2.3421754037750544, -2.342175403774924, com)
ilam = verifyeig(A, λ[i], x[:, i], B) = Interval{Float64}(-1.8204436211401773, -1.820443621140108, com)
ilam = verifyeig(A, λ[i], x[:, i], B) = Interval{Float64}(-0.9825359650919452, -0.9825359650918856, com) + Interval{Float64}(0.9062641005675408, 0.9062641005675977, com)im
ilam = verifyeig(A, λ[i], x[:, i], B) = Interval{Float64}(-0.9825359650919446, -0.9825359650918858, com) + Interval{Float64}(-0.9062641005675972, -0.9062641005675407, com)im
ilam = verifyeig(A, λ[i], x[:, i], B) = Interval{Float64}(-0.9369434243815297, -0.9369434243814961, com)
ilam = verifyeig(A, λ[i], x[:, i], B) = Interval{Float64}(-0.7678548617673198, -0.7678548617672895, com)
ilam = verifyeig(A, λ[i], x[:, i], B) = Interval{Float64}(-0.6336042505435233, -0.6336042505432147, com) + Interval{Float64}(-4.458

**TODO**　固有値が縮退し多重固有値である場合や複数の固有値がクラスター形成している場合、この方法は失敗する。この場合の精度保証付き数値計算方法は文献3にある。今後この方法を実装していきたい。


本資料は以下のような文献・Web ページ等を参考に書いている。

### 参考文献

1. 大石進一編著, 精度保証付き数値計算の基礎, コロナ社, 2018.<br>
（精度保証付き数値計算の教科書、3.4.2章にある「非線形方程式を利用した精度保証法」を実装した）

1. S.M. Rump. Guaranteed Inclusions for the Complex Generalized Eigenproblem. Computing, 42:225-238, 1989.<br>
（今回の方法を初めて発表した原著論文）

1. S.M. Rump. Computational Error Bounds for Multiple or Nearly Multiple Eigenvalues. Linear Algebra and its Applications (LAA), 324:209–226, 2001.<br>
（多重固有値や多数の固有値のクラスターに対する精度保証付き数値計算方法。INTLABの`verifyeig.m`にはこの方法も実装されている）


<div align="right">近藤慎佑, <a href="http://www.risk.tsukuba.ac.jp/~takitoshi/">高安亮紀</a>，2022年9月19日（最終更新：2024年2月5日）</div>